In [1]:
import pandas as pd
import pyodbc
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil.rrule import rrule, MONTHLY
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import plotly.express as px
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import shap
import numdifftools as nd
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope
shap.initjs()
%load_ext autoreload
%autoreload 2
import mmm_transformations
import mmm_preprocessing
import mmm_modeling
import mmm_response_curves
import mmm_optimization

# Summary

In [223]:
# description of what this notebook does
# should be minimal code and jsut outputs

# Data Pull

In [2]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRDSPPI10DB1;'
                      'Port=1433;')

In [3]:
sql_stmt = """SELECT	r.HCOId,
		hcoAddr.BaseZip AS HCOZip,
		r.MonthDesc,
		r.YearDesc,
		r.BrandName,
		r.Emails,
		r.Phones,
		r.FTF,
		r.Virtual,
		r.Mssg,
		r.Other,
		r.Quantity,
		fncl.*,
		hco_bed_total.*,
		hco_bus.*

FROM (
SELECT	ISNULL(a.HCOId, b.HCOId) AS HCOId,
		ISNULL(a.YearDesc, b.YearDesc) AS YearDesc,
		ISNULL(a.MonthDesc, b.MonthDesc) AS MonthDesc,
		ISNULL(a.BrandName, b.BrandName) AS BrandName,
		ISNULL(b.Emails, 0) AS Emails,
		ISNULL(b.Phones, 0) AS Phones,
		ISNULL(b.FTF, 0) AS FTF,
		ISNULL(b.Virtual, 0) AS Virtual,
		ISNULL(b.Mssg, 0) AS Mssg,
		ISNULL(b.Other, 0) AS Other,
		ISNULL(a.Quantity, 0) AS Quantity 
FROM (
	SELECT	s.HCOId, 
		d.YearDesc, 
		d.MonthDesc,
		p.BrandName,
		SUM(s.QuantityAdjusted) AS Quantity		
		FROM meas.tblFactSalesTransaction s
		INNER JOIN meas.tblDimDate d ON s.SalesTransactionDateId = d.DateId
		INNER JOIN	meas.tblDimProduct p	ON	s.ProductId = p.ProductId
		WHERE s.HCOId <> -1
		GROUP BY s.HCOId, 
				d.YearDesc, 
				d.MonthDesc,
				p.BrandName
) a
/* ProductId =1 (Rolvedon). All ProductId in FasctSalesTrasaction equals to 1 
 FactSalesTransaction (hco-date level) has 2819 rows and
 a (hco-month level) has 945 rows*/ 


FULL OUTER JOIN (
SELECT
    fc.HCOId, 
    d.YearDesc,
    d.MonthDesc,
    p.BrandName,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Email_vod' THEN fc.CallId END) AS Emails,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Phone_vod' THEN fc.CallId END) AS Phones,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Face_to_face_vod' THEN fc.CallId END) AS FTF,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Video_vod' THEN fc.CallId END) AS Virtual,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Message_vod' THEN fc.CallId END) AS Mssg,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Other_vod' THEN fc.CallId END) AS Other
FROM
    (
        SELECT * 
		/*
            CASE
                WHEN HCPId <> -1 THEN HCPId
                ELSE HCOId
            END AS ChildId,
			CASE 
				WHEN HCPId <> -1 THEN 1
				ELSE 2
			END AS AffiliationTypeId */
        FROM meas.tblFactCall
		WHERE HCOId <> -1
/* It is confirmed that FactCAll is either HCPId or HCOId level 
HCOId = -1 has 8173 rows; HCO<>-1 has 5685. tblFactCall has a total of 13858 rows*/         
    ) fc
/*LEFT JOIN
    meas.tblDimAffiliation da ON fc.ChildId = da.ChildId AND fc.AffiliationTypeId=da.AffiliationTypeId and da.[Primary]=1 and da.Active=1
	 type=1, HCP-HCO, type=2, HCO-HCO  */

INNER JOIN
    dbo.tblDimCallChannel cc ON fc.CallChannelId = cc.CallChannelId
INNER JOIN
    meas.tblDimDate d ON fc.CallDateId = d.DateId
INNER JOIN
    meas.tblFactCallDetail fcd ON fc.CallSourceId = fcd.CallSourceId
LEFT JOIN
    meas.tblDimProduct p ON fcd.ProductId = p.ProductId
GROUP BY
    fc.HCOId,
    d.YearDesc,
    d.MonthDesc,
    p.BrandName
) b 
/*FactCall has 13858 (date-level) rows and b (month level) has 2885 rows*/
ON a.HCOId = b.HCOId AND a.YearDesc = b.YearDesc
	AND a.MonthDesc = b.MonthDesc
	AND a.BrandName = b.BrandName
) r
INNER JOIN MEAS.tblDimHCO hco ON r.HCOId = hco.HCOId
LEFT OUTER JOIN 
(SELECT TPSEntityId as HCOSourceId, iq_hco_fncl.*
FROM [SPECTRUM_ADHOC].[dbo].[tblDataFeed_IQVIA_OneKey_HCO_FNCL_FACT] iq_hco_fncl
--tblDataFeed_IQVIA_OneKey_HCO_FNCL_FACT has 4149 rows
INNER JOIN (
    SELECT *
    FROM [SPECTRUM_CM].[CM].[tblOutletAlternateId]
    WHERE TPSDataFeedId = 6
) oa ON iq_hco_fncl.HCO_HCE_ID = oa.DataProviderUniqueIdentifier COLLATE SQL_Latin1_General_CP1_CI_AS
-- "SQL_Latin1_General_CP1_CI_AS" indicates to read collumn via "class insensitive"
  ) fncl ON hco.HCOSourceID = fncl.HCOSourceId 
--fncl only has 4149 rows

LEFT OUTER JOIN 
(SELECT TPSEntityId as HCOSourceId, bed.*
FROM [SPECTRUM_ADHOC].[dbo].[tblDataFeed_IQVIA_OneKey_HCO_BED_FACT] bed 
INNER JOIN (
    SELECT *
    FROM [SPECTRUM_CM].[CM].[tblOutletAlternateId]
    WHERE TPSDataFeedId = 6
) oa ON bed.HCO_HCE_ID = oa.DataProviderUniqueIdentifier COLLATE SQL_Latin1_General_CP1_CI_AS
WHERE BED_DESC = 'Total' 
  ) hco_bed_total ON hco.HCOSourceID = hco_bed_total.HCOSourceId 
 --hco_bed_total has 4725 rows


LEFT OUTER JOIN 
(SELECT TPSEntityId as HCOSourceId, bus.* FROM (SELECT
    HCO_HCE_ID,
    MAX(CASE WHEN DETL_CD = 'avgoccrate' THEN DETL_NBR END) AS avgoccrate,
    MAX(CASE WHEN DETL_CD = 'routineday' THEN DETL_NBR END) AS routineday,
    MAX(CASE WHEN DETL_CD = 'H_pdays' THEN DETL_NBR END) AS H_pdays,
    MAX(CASE WHEN DETL_CD = 'avglos' THEN DETL_NBR END) AS avglos,
    MAX(CASE WHEN DETL_CD = 'mparmix' THEN DETL_NBR END) AS mparmix,
    MAX(CASE WHEN DETL_CD = 'mcaredrg' THEN DETL_NBR END) AS mcaredrg,
    MAX(CASE WHEN DETL_CD = 'ttldrg' THEN DETL_NBR END) AS ttldrg,
    MAX(CASE WHEN DETL_CD = 'MCR_alos' THEN DETL_NBR END) AS MCR_alos,
    MAX(CASE WHEN DETL_CD = 'hdrg' THEN DETL_NBR END) AS hdrg,
    MAX(CASE WHEN DETL_CD = 'MCR_pdays' THEN DETL_NBR END) AS MCR_pdays
FROM
    [SPECTRUM_ADHOC].[dbo].[tblDataFeed_IQVIA_OneKey_HCO_BUS_DETL_FACT]
GROUP BY
    HCO_HCE_ID
HAVING
    COALESCE(MAX(CASE WHEN DETL_CD = 'avgoccrate' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'routineday' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'H_pdays' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'avglos' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'mparmix' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'mcaredrg' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'ttldrg' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'MCR_alos' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'hdrg' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'MCR_pdays' THEN DETL_NBR END)) IS NOT NULL
) bus
INNER JOIN (
    SELECT *
    FROM [SPECTRUM_CM].[CM].[tblOutletAlternateId]
    WHERE TPSDataFeedId = 6
) oa ON bus.HCO_HCE_ID = oa.DataProviderUniqueIdentifier COLLATE SQL_Latin1_General_CP1_CI_AS
) hco_bus ON hco.HCOSourceID = hco_bus.HCOSourceId 
--hco_bus has 4145 rows 

LEFT OUTER JOIN MEAS.tblDimEntityAddress hcoEntAddr
	ON hco.HCOId = hcoEntAddr.EntityId 
	AND hcoEntAddr.EntityType = 'HCO'
	AND hcoEntAddr.Active = 1
	AND hcoEntAddr.IsPrimary = 1
LEFT OUTER JOIN MEAS.tblDimAddress hcoAddr
	ON hcoEntAddr.AddressId = hcoAddr.AddressId
	AND hcoAddr.Active = 1
ORDER BY r.YearDesc DESC, r.MonthDesc DESC

/* MEAS.tblDimHCO has 53747 distinct id 
using the latest b query, the output of this query is now 8182 rows. (0719)
using the latest b query (consider only hco), the output of this query is now 3845 rows
using the latest b query (consider only hco) + fncl, the output of this query is now 290 rows */ 
"""

In [4]:
df = pd.read_sql(sql_stmt, conn)
df

DatabaseError: Execution failed on sql 'SELECT r.HCOId,
                     hcoAddr.BaseZip AS HCOZip,
                     r.MonthDesc,
                     r.YearDesc,
                     r.ProductGroupName,
                     r.Emails,
                     r.Phones,
                     r.FTF,
                     r.Virtual,
                     r.Mssg,
                     r.Other,
                     r.Quantity

              FROM (
              SELECT  ISNULL(a.HCOId, b.HCOId) AS HCOId,
                      ISNULL(a.YearDesc, b.YearDesc) AS YearDesc,
                      ISNULL(a.MonthDesc, b.MonthDesc) AS MonthDesc,
                      ISNULL(a.ProductGroupName, b.ProductGroupName) AS ProductGroupName,
                      ISNULL(b.Emails, 0) AS Emails,
                      ISNULL(b.Phones, 0) AS Phones,
                      ISNULL(b.FTF, 0) AS FTF,
                      ISNULL(b.Virtual, 0) AS Virtual,
                      ISNULL(b.Mssg, 0) AS Mssg,
              

# Preprocessing

In [109]:
df = pd.read_csv('spectrum_balanced.csv')
df

,SystemID,Date,Emails,Phones,FTF,Virtual,Mssg,Other,Quantity,DDS_IN_PD_CNT,...,avgoccrate,routineday,H_pdays,avglos,mparmix,mcaredrg,ttldrg,MCR_alos,hdrg,MCR_pdays
0,4769,2022-10-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
1,4769,2022-11-01,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
2,4769,2022-12-01,0.0,1.0,1.0,0.0,0.0,0.0,150.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
3,4769,2023-01-01,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
4,4769,2023-02-01,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,44280,2023-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
8756,44280,2023-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
8757,44280,2023-05-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
8758,44280,2023-06-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04


In [110]:
len(np.unique(df['SystemID']))

876

In [112]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
SystemID,8760.0,2.861339e+04,1.005378e+04,4746.0000,2.288075e+04,27075.50,32540.00,5.405300e+04
Emails,8760.0,5.489726e-01,7.811360e+00,0.0000,0.000000e+00,0.00,0.00,5.940000e+02
Phones,8760.0,2.327626e-01,3.156503e+00,0.0000,0.000000e+00,0.00,0.00,1.980000e+02
FTF,8760.0,3.553653e-01,1.188258e+01,0.0000,0.000000e+00,0.00,0.00,9.240000e+02
Virtual,8760.0,3.721461e-02,7.482421e-01,0.0000,0.000000e+00,0.00,0.00,6.600000e+01
Mssg,8760.0,1.392694e-02,1.863908e-01,0.0000,0.000000e+00,0.00,0.00,6.000000e+00
Other,8760.0,7.157534e-02,3.331449e-01,0.0000,0.000000e+00,0.00,0.00,6.000000e+00
Quantity,8760.0,4.307192e+00,7.212931e+01,0.0000,0.000000e+00,0.00,0.00,4.830000e+03
DDS_IN_PD_CNT,8760.0,2.479339e+03,3.015316e+03,183.0000,1.456000e+03,2479.34,2479.34,4.208000e+04
INPAT_REV_AMT,8760.0,4.653628e+06,7.319108e+06,1351.0000,2.875990e+06,4653627.87,4653627.87,1.713868e+08


In [113]:
df_pp = df.copy()
df_pp.columns = df_pp.columns.str.lower()
df_pp

,systemid,date,emails,phones,ftf,virtual,mssg,other,quantity,dds_in_pd_cnt,...,avgoccrate,routineday,h_pdays,avglos,mparmix,mcaredrg,ttldrg,mcr_alos,hdrg,mcr_pdays
0,4769,2022-10-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
1,4769,2022-11-01,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
2,4769,2022-12-01,0.0,1.0,1.0,0.0,0.0,0.0,150.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
3,4769,2023-01-01,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
4,4769,2023-02-01,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,44280,2023-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
8756,44280,2023-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
8757,44280,2023-05-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04
8758,44280,2023-06-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,2.67,233495.55,233495.55,37.93,2.77,14743.03,57472.57,39.97,57472.57,69276.04


# Transformations

In [117]:
transform = mmm_transformations.MMMTransformations()

In [121]:
df_pp.describe()

,systemid,emails,phones,ftf,virtual,mssg,other,quantity,dds_in_pd_cnt,inpat_rev_amt,...,avgoccrate,routineday,h_pdays,avglos,mparmix,mcaredrg,ttldrg,mcr_alos,hdrg,mcr_pdays
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8.760000e+03,...,8760.000000,8.760000e+03,8.760000e+03,8760.000000,8760.000000,8760.000000,8.760000e+03,8760.000000,8.760000e+03,8.760000e+03
mean,28613.390411,0.548973,0.232763,0.355365,0.037215,0.013927,0.071575,4.307192,2479.339292,4.653628e+06,...,2.669699,2.334956e+05,2.334956e+05,37.931456,2.770395,14743.031952,5.747257e+04,39.971473,5.747257e+04,6.927604e+04
std,10053.784079,7.811360,3.156503,11.882578,0.748242,0.186391,0.333145,72.129307,3015.315842,7.319108e+06,...,3.237925,2.789794e+05,2.789794e+05,64.042464,3.432349,17637.936518,7.163012e+04,56.384977,7.163012e+04,8.004844e+04
min,4746.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,183.000000,1.351000e+03,...,0.036300,5.530000e+02,5.530000e+02,1.522800,0.003700,5.000000,7.600000e+01,1.000000,7.600000e+01,1.700000e+01
25%,22880.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1456.000000,2.875990e+06,...,1.498300,1.788200e+05,1.788200e+05,18.733000,1.440625,10556.750000,4.252125e+04,22.000000,4.252125e+04,4.825275e+04
50%,27075.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2479.340000,4.653628e+06,...,2.670000,2.334955e+05,2.334955e+05,37.930000,2.770000,14743.030000,5.747257e+04,39.970000,5.747257e+04,6.927604e+04
75%,32540.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2479.340000,4.653628e+06,...,2.670000,2.334955e+05,2.334955e+05,37.930000,2.770000,14743.030000,5.747257e+04,39.970000,5.747257e+04,6.927604e+04
max,54053.000000,594.000000,198.000000,924.000000,66.000000,6.000000,6.000000,4830.000000,42080.000000,1.713868e+08,...,50.049800,5.414923e+06,5.414923e+06,1238.712500,49.508900,334041.000000,1.424002e+06,1069.000000,1.424002e+06,1.466733e+06


In [166]:
df_t = transform.winsorize(df_pp, ['quantity', 'emails', 'phones', 'ftf', 'virtual', 'mssg', 'other'], 99.7)
df_t.describe()

,systemid,emails,phones,ftf,virtual,mssg,other,quantity,dds_in_pd_cnt,inpat_rev_amt,...,avgoccrate,routineday,h_pdays,avglos,mparmix,mcaredrg,ttldrg,mcr_alos,hdrg,mcr_pdays
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8.760000e+03,...,8760.000000,8.760000e+03,8.760000e+03,8760.000000,8760.000000,8760.000000,8.760000e+03,8760.000000,8.760000e+03,8.760000e+03
mean,28613.390411,0.382648,0.152685,0.125571,0.026256,0.011416,0.067123,2.421618,2479.339292,4.653628e+06,...,2.669699,2.334956e+05,2.334956e+05,37.931456,2.770395,14743.031952,5.747257e+04,39.971473,5.747257e+04,6.927604e+04
std,10053.784079,1.431440,0.687101,0.563156,0.188116,0.132966,0.291942,17.494849,3015.315842,7.319108e+06,...,3.237925,2.789794e+05,2.789794e+05,64.042464,3.432349,17637.936518,7.163012e+04,56.384977,7.163012e+04,8.004844e+04
min,4746.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,183.000000,1.351000e+03,...,0.036300,5.530000e+02,5.530000e+02,1.522800,0.003700,5.000000,7.600000e+01,1.000000,7.600000e+01,1.700000e+01
25%,22880.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1456.000000,2.875990e+06,...,1.498300,1.788200e+05,1.788200e+05,18.733000,1.440625,10556.750000,4.252125e+04,22.000000,4.252125e+04,4.825275e+04
50%,27075.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2479.340000,4.653628e+06,...,2.670000,2.334955e+05,2.334955e+05,37.930000,2.770000,14743.030000,5.747257e+04,39.970000,5.747257e+04,6.927604e+04
75%,32540.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2479.340000,4.653628e+06,...,2.670000,2.334955e+05,2.334955e+05,37.930000,2.770000,14743.030000,5.747257e+04,39.970000,5.747257e+04,6.927604e+04
max,54053.000000,16.000000,7.723000,6.000000,2.000000,2.000000,2.000000,223.014000,42080.000000,1.713868e+08,...,50.049800,5.414923e+06,5.414923e+06,1238.712500,49.508900,334041.000000,1.424002e+06,1069.000000,1.424002e+06,1.466733e+06


In [168]:
df_t.to_csv('spectrum_balanced_winsorize.csv', index=False)

In [169]:
df_t = transform.lag_dv(df_t, 'quantity', 3, 'systemid')
df_t = transform.lag_dv(df_t, 'emails', 3, 'systemid')
df_t = transform.lag_dv(df_t, 'phones', 3, 'systemid')
df_t = transform.lag_dv(df_t, 'ftf', 3, 'systemid')
df_t = transform.lag_dv(df_t, 'virtual', 3, 'systemid')
df_t = transform.lag_dv(df_t, 'mssg', 3, 'systemid')
df_t = transform.lag_dv(df_t, 'other', 3, 'systemid')
df_t

,systemid,date,emails,phones,ftf,virtual,mssg,other,quantity,dds_in_pd_cnt,...,ftf_lag3,virtual_lag1,virtual_lag2,virtual_lag3,mssg_lag1,mssg_lag2,mssg_lag3,other_lag1,other_lag2,other_lag3
0,4769,2022-10-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4769,2022-11-01,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2479.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4769,2022-12-01,0.0,1.0,1.0,0.0,0.0,0.0,150.0,2479.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4769,2023-01-01,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4769,2023-02-01,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,44280,2023-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8756,44280,2023-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8757,44280,2023-05-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8758,44280,2023-06-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2479.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [170]:
df_t.corr()

,systemid,emails,phones,ftf,virtual,mssg,other,quantity,dds_in_pd_cnt,inpat_rev_amt,...,ftf_lag3,virtual_lag1,virtual_lag2,virtual_lag3,mssg_lag1,mssg_lag2,mssg_lag3,other_lag1,other_lag2,other_lag3
systemid,1.000000,0.031459,0.001065,-0.005450,0.019553,0.029717,-0.001924,-0.013455,-0.010496,-0.002404,...,-0.000133,0.014906,0.012526,0.013923,0.032622,0.034322,0.031844,0.000756,0.013295,0.017943
emails,0.031459,1.000000,0.478570,0.447268,0.323069,0.067023,0.023769,0.377197,0.049966,0.026759,...,0.295146,0.176220,0.145056,0.111125,0.061231,0.048892,0.078128,0.016383,0.011291,0.049344
phones,0.001065,0.478570,1.000000,0.406383,0.206148,0.064646,0.004933,0.279862,0.050885,0.017265,...,0.223331,0.120111,0.078673,0.068575,0.027611,0.021022,0.029078,0.010516,0.004624,0.014918
ftf,-0.005450,0.447268,0.406383,1.000000,0.159625,0.064712,-0.010997,0.314439,0.038547,0.015833,...,0.271094,0.090156,0.087169,0.055970,0.058663,0.029121,0.062928,0.007041,0.014865,0.008132
virtual,0.019553,0.323069,0.206148,0.159625,1.000000,0.051917,0.030271,0.091101,0.041529,0.021507,...,0.117177,0.079780,0.081000,0.060006,0.052165,0.035042,0.047333,0.027117,-0.013818,0.037088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mssg_lag2,0.034322,0.048892,0.021022,0.029121,0.035042,0.376721,-0.009317,0.001839,0.007723,0.003505,...,0.035994,0.070099,0.094890,0.008675,0.377125,1.000000,0.335814,-0.007864,0.007957,-0.007023
mssg_lag3,0.031844,0.078128,0.029078,0.062928,0.047333,0.296696,0.013128,0.027808,0.002216,-0.000004,...,0.064433,0.045912,0.082772,0.111481,0.355171,0.335814,1.000000,-0.014283,-0.012373,0.015704
other_lag1,0.000756,0.016383,0.010516,0.007041,0.027117,-0.015406,0.167387,-0.017694,0.121562,0.100117,...,0.018357,0.038726,0.027129,0.021719,0.006063,-0.007864,-0.014283,1.000000,0.195626,0.111474
other_lag2,0.013295,0.011291,0.004624,0.014865,-0.013818,-0.009103,0.094535,-0.019932,0.099750,0.076911,...,0.013343,0.036831,0.039547,0.029905,-0.011185,0.007957,-0.012373,0.195626,1.000000,0.205404


# Final Model Fitting

In [171]:
modeling = mmm_modeling.MMMModeling()

In [172]:
channels = ['emails', 'phones', 'ftf', 'virtual', 'mssg', 'other']
lag_dv = [x for x in df_t.columns if 'quantity_lag' in x]
lag_channels = [x for x in df_t.columns if ('lag' in x) & ('quantity' not in x)]
non_media = df_t.drop(channels + lag_dv + lag_channels + ['quantity', 'systemid', 'date'], axis=1).columns.tolist()
#X = df_t[channels + lag_channels + lag_dv]
X = df_t[channels + lag_channels + lag_dv + non_media]
y = df_t['quantity']
model = modeling.rf_regressor(df_t, X.columns.tolist(), 'quantity', 'date')

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_modeling.py:62: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_modeling.py:63: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [173]:
# performance
model['performance']

{'full': {'r2': 0.767229648602483,
  'rmse': 8.44013204365791,
  'mape': 0.6753667355849348},
 'train': {'r2': 0.712072759080965,
  'rmse': 8.663354419713025,
  'mape': 0.7219950161670617},
 'test': {'r2': 0.21732141175783048,
  'rmse': 19.411826164156786,
  'mape': 1.6505982211740842}}

In [174]:
# importance
model['importance'].head(50)

,feature,importance,std
26,quantity_lag3,0.316919,0.078330
24,quantity_lag1,0.172119,0.055458
0,emails,0.078377,0.048849
2,ftf,0.059888,0.034220
25,quantity_lag2,0.059685,0.034441
1,phones,0.043026,0.022404
6,emails_lag1,0.040154,0.027530
8,emails_lag3,0.031891,0.021535
7,emails_lag2,0.028149,0.020198
13,ftf_lag2,0.026658,0.022477


# Response Curves

In [134]:
response_curves = mmm_response_curves.MMMResponseCurves()

In [175]:
channels = ['emails', 'phones', 'ftf', 'virtual', 'mssg', 'other']
lag_dv = [x for x in df_t.columns if 'quantity_lag' in x]
lag_channels = [x for x in df_t.columns if ('lag' in x) & ('quantity' not in x)]
non_media = df_t.drop(channels + lag_dv + lag_channels + ['quantity', 'systemid', 'date'], axis=1).columns.tolist()
X = df_t[channels + lag_channels + lag_dv + non_media]

In [159]:
X.describe()

,emails,phones,ftf,virtual,mssg,other,emails_lag1,emails_lag2,emails_lag3,phones_lag1,...,avgoccrate,routineday,h_pdays,avglos,mparmix,mcaredrg,ttldrg,mcr_alos,hdrg,mcr_pdays
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,...,8760.000000,8.760000e+03,8.760000e+03,8760.000000,8760.000000,8760.000000,8.760000e+03,8760.000000,8.760000e+03,8.760000e+03
mean,0.424582,0.173421,0.142524,0.028196,0.012557,0.070339,0.365623,0.271394,0.221512,0.142001,...,2.669699,2.334956e+05,2.334956e+05,37.931456,2.770395,14743.031952,5.747257e+04,39.971473,5.747257e+04,6.927604e+04
std,2.067320,1.002271,0.809238,0.212100,0.152843,0.318081,1.936358,1.572809,1.318773,0.917224,...,3.237925,2.789794e+05,2.789794e+05,64.042464,3.432349,17637.936518,7.163012e+04,56.384977,7.163012e+04,8.004844e+04
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.036300,5.530000e+02,5.530000e+02,1.522800,0.003700,5.000000,7.600000e+01,1.000000,7.600000e+01,1.700000e+01
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.498300,1.788200e+05,1.788200e+05,18.733000,1.440625,10556.750000,4.252125e+04,22.000000,4.252125e+04,4.825275e+04
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.670000,2.334955e+05,2.334955e+05,37.930000,2.770000,14743.030000,5.747257e+04,39.970000,5.747257e+04,6.927604e+04
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.670000,2.334955e+05,2.334955e+05,37.930000,2.770000,14743.030000,5.747257e+04,39.970000,5.747257e+04,6.927604e+04
max,42.482000,20.241000,15.723000,3.000000,3.000000,3.241000,42.482000,42.482000,42.482000,20.241000,...,50.049800,5.414923e+06,5.414923e+06,1238.712500,49.508900,334041.000000,1.424002e+06,1069.000000,1.424002e+06,1.466733e+06


In [208]:
channel1 = response_curves.responses(model['full_model'], X, 'emails', 20, 0.1)

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

In [209]:
response_curves.plot(channel1['resp_df'], 'touches', ['emails', 'emails_hill_estimate'])

In [210]:
channel1['resp_df'].to_csv('spectrum_emails.csv', index=False)

In [184]:
channel2 = response_curves.responses(model['full_model'], X, 'phones', 20, 0.1)

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

In [191]:
response_curves.plot(channel2['resp_df'], 'touches', ['phones', 'phones_hill_estimate'])

In [197]:
channel2['resp_df'].to_csv('spectrum_phones.csv', index=False)

In [187]:
channel3 = response_curves.responses(model['full_model'], X, 'ftf', 20, 0.1)

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh

In [188]:
response_curves.plot(channel3['resp_df'], 'touches', ['ftf', 'ftf_hill_estimate'])

In [198]:
channel3['resp_df'].to_csv('spectrum_ftf.csv', index=False)

In [199]:
channel4 = response_curves.responses(model['full_model'], X, 'virtual', 20, 0.1)

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

In [200]:
response_curves.plot(channel4['resp_df'], 'touches', ['virtual', 'virtual_hill_estimate'])

In [201]:
channel4['resp_df'].to_csv('spectrum_virtual.csv', index=False)

In [202]:
channel5 = response_curves.responses(model['full_model'], X, 'mssg', 20, 0.1)

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

In [203]:
response_curves.plot(channel5['resp_df'], 'touches', ['mssg', 'mssg_hill_estimate'])

In [204]:
channel5['resp_df'].to_csv('spectrum_mssg.csv', index=False)

In [205]:
channel6 = response_curves.responses(model['full_model'], X, 'other', 20, 0.1)

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 ... (more hidden) ...D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the res

In [206]:
response_curves.plot(channel6['resp_df'], 'touches', ['other', 'other_hill_estimate'])

In [207]:
channel6['resp_df'].to_csv('spectrum_other.csv', index=False)

# Channel 1 Optimization

In [57]:
p1_hill = pd.DataFrame(channel1_segment['optimal_hill']).T.reset_index()
p1_hill.columns = ['feature', 'beta', 'gamma', 'alpha']
p1_hill

,feature,beta,gamma,alpha
0,Specialty_AC_P1_Arikayce,205.637938,51.313808,4.712360
1,Specialty_ADU_P1_Arikayce,205.517174,51.329996,4.719966
2,Specialty_CCE_P1_Arikayce,204.495296,51.466856,4.785168
3,Specialty_CCM_P1_Arikayce,184.168059,53.853232,6.000614
4,Specialty_EM_P1_Arikayce,205.726333,51.301955,4.706769
5,Specialty_FM_P1_Arikayce,204.875774,51.678124,5.028733
6,Specialty_GP_P1_Arikayce,203.352425,51.619572,4.858920
7,Specialty_HOS_P1_Arikayce,205.886561,51.280453,4.696636
8,Specialty_ID_P1_Arikayce,234.191120,48.921949,5.973733
9,Specialty_IM_P1_Arikayce,207.176496,49.692169,3.784883


In [58]:
# hyperopt hill
optimization = mmm_optimization.MMMOptimization(budget=354, params=p1_hill)
channels = p1_hill['feature'].tolist()
output = optimization.optimize_hyperopt_hill(channels, 2000)
output

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:10<00:00, 28.31trial/s, best loss: 99997133.878953]


{'mix': {'Specialty_AC_P1_Arikayce': 330,
  'Specialty_ADU_P1_Arikayce': 223,
  'Specialty_CCE_P1_Arikayce': 194,
  'Specialty_CCM_P1_Arikayce': 340,
  'Specialty_EM_P1_Arikayce': 174,
  'Specialty_FM_P1_Arikayce': 171,
  'Specialty_GP_P1_Arikayce': 250,
  'Specialty_HOS_P1_Arikayce': 222,
  'Specialty_ID_P1_Arikayce': 266,
  'Specialty_IM_P1_Arikayce': 211,
  'Specialty_Other_P1_Arikayce': 330,
  'Specialty_PCC_P1_Arikayce': 188,
  'Specialty_PDP_P1_Arikayce': 252,
  'Specialty_PUD_P1_Arikayce': 221},
 'trials': [{'loss': 99997911.78291155, 'status': 'ok'},
  {'loss': 99997367.74888934, 'status': 'ok'},
  {'loss': 99997252.43217513, 'status': 'ok'},
  {'loss': 99997818.15737288, 'status': 'ok'},
  {'loss': 99997880.892491, 'status': 'ok'},
  {'loss': 99997516.23347887, 'status': 'ok'},
  {'loss': 99997876.8572574, 'status': 'ok'},
  {'loss': 99997331.26427692, 'status': 'ok'},
  {'loss': 99997140.25295413, 'status': 'ok'},
  {'loss': 99997584.77514112, 'status': 'ok'},
  {'loss': 9999

# ID Budgeting

In [71]:
optimization = mmm_optimization.MMMOptimization(budget=135)
channels = ['P1_Arikayce', 'P2_Arikayce']
specialties = [x for x in df_t.columns if x.startswith('Specialty_')]
lag_dv = [x for x in df_t.columns if 'rx_count_lag' in x]
lag_channels = [x for x in df_t.columns if ('lag' in x) & ('rx_count' not in x)]
X = df_t[channels + lag_channels + specialties]
X[specialties] = 0
X['Specialty_ID'] = 1
output = optimization.optimize_predict(X, channels, 2000, model['full_model'])
output

  0%|                                                                                                                             | 0/2000 [00:00<?, ?trial/s, best loss=?]

D:\Users\hartsingh\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:3645: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\program files (x86)\python\python37-32\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:02<00:00, 31.98trial/s, best loss: -264.04118068466823]


{'mix': {'P1_Arikayce': 90, 'P2_Arikayce': 3},
 'trials': [{'loss': 99999740.31881931, 'status': 'ok'},
  {'loss': -123.15118068466835, 'status': 'ok'},
  {'loss': 99999738.89881931, 'status': 'ok'},
  {'loss': 99999785.74881932, 'status': 'ok'},
  {'loss': -34.57118068466833, 'status': 'ok'},
  {'loss': -126.65118068466835, 'status': 'ok'},
  {'loss': -6.371180684668324, 'status': 'ok'},
  {'loss': -260.7011806846684, 'status': 'ok'},
  {'loss': 99999740.31881931, 'status': 'ok'},
  {'loss': 99999740.65881932, 'status': 'ok'},
  {'loss': -123.15118068466835, 'status': 'ok'},
  {'loss': -11.851180684668325, 'status': 'ok'},
  {'loss': 99999732.60881932, 'status': 'ok'},
  {'loss': 99999741.44881931, 'status': 'ok'},
  {'loss': 99999732.56881931, 'status': 'ok'},
  {'loss': -19.181180684668327, 'status': 'ok'},
  {'loss': -124.69118068466831, 'status': 'ok'},
  {'loss': 99999872.34881932, 'status': 'ok'},
  {'loss': 99999740.15881932, 'status': 'ok'},
  {'loss': 99999952.48881932, 'stat

# Overall Budget

In [74]:
optimization = mmm_optimization.MMMOptimization(budget=368)
channels = ['P1_Arikayce', 'P2_Arikayce']
specialties = [x for x in df_t.columns if x.startswith('Specialty_')]
lag_dv = [x for x in df_t.columns if 'rx_count_lag' in x]
lag_channels = [x for x in df_t.columns if ('lag' in x) & ('rx_count' not in x)]
X = df_t[channels + lag_channels + specialties]
output = optimization.optimize_predict(X, channels, 2000, model['full_model'])
output

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:01<00:00, 32.58trial/s, best loss: -271.404799150921]


{'mix': {'P1_Arikayce': 288, 'P2_Arikayce': 13},
 'trials': [{'loss': 99999736.8873437, 'status': 'ok'},
  {'loss': 99999778.46091513, 'status': 'ok'},
  {'loss': -268.7090848652068, 'status': 'ok'},
  {'loss': -255.94622772234968, 'status': 'ok'},
  {'loss': -221.10908486520682, 'status': 'ok'},
  {'loss': 99999758.573058, 'status': 'ok'},
  {'loss': 99999784.95734371, 'status': 'ok'},
  {'loss': 99999743.05377227, 'status': 'ok'},
  {'loss': 99999743.05377227, 'status': 'ok'},
  {'loss': -8.623370579492526, 'status': 'ok'},
  {'loss': -195.39979915092113, 'status': 'ok'},
  {'loss': 99999790.75448656, 'status': 'ok'},
  {'loss': -262.32837057949257, 'status': 'ok'},
  {'loss': -33.75122772234967, 'status': 'ok'},
  {'loss': 99999736.60591513, 'status': 'ok'},
  {'loss': -208.24551343663535, 'status': 'ok'},
  {'loss': 99999743.05377227, 'status': 'ok'},
  {'loss': 99999743.05377227, 'status': 'ok'},
  {'loss': -218.0883705794926, 'status': 'ok'},
  {'loss': -207.00408486520678, 'stat

In [ ]:
# dont need lag dv - point is to estimate the impact of channels as best as possible and that happens when their importance is high